In [1]:
import oracledb

In [2]:
conn_nwp = oracledb.connect(user="nwp", password="modeli", dsn="192.168.211.62/serv")
cursor = conn_nwp.cursor()

cursor.execute("select * from model_num")
for c in cursor:
    print(c)

In [5]:
conn_173 = oracledb.connect(user="cligts", password="belorgmc", dsn="192.168.211.173/clidb")
cur_173 = conn_173.cursor()

cur_173.execute("select date_obs from gts.synop where date_obs = '22.12.2023' and station_id='06450'")
for c in cur_173:
    print(c)

OperationalError: DPY-6005: cannot connect to database (CONNECTION_ID=rm8o/mrB+zO7V4fKjShPOA==).
DPY-4011: the database or network closed the connection
Help: https://python-oracledb.readthedocs.io/en/latest/user_guide/troubleshooting.html#dpy-4011

(92, 'WRF_92                        ', '15 km                         ', 'WRF-92                        ')
(93, 'WRF_93                        ', '15 km                         ', 'WRF-93                        ')
(94, 'WRF_94                        ', '15 km                         ', 'WRF-94                        ')
(95, 'WRF_95                        ', '15 km                         ', 'WRF-95                        ')
(982, 'GEO_982                       ', '3 km                          ', 'GEO-982                       ')
(983, 'GEO_983                       ', '3 km                          ', 'GEO-983                       ')
(984, 'GEO_984                       ', '3 km                          ', 'GEO-984                       ')
(985, 'GEO_985                       ', '3 km                          ', 'GEO-985                       ')
(3, 'UM_1.0                        ', '1.0°                         ', 'UM                            ')
(4, 'GFS_0.25                      